# Effective bounds on H<sub>t</sub>

The following formulae are cut and pasted as LaTeX from [Effective bounds on H_t - second approach](http://michaelnielsen.org/polymath1/index.php?title=Effective_bounds_on_H_t_-_second_approach). They are presented in a different order, one which facilitates coding. The object is to implement $E_1$, $E_2$, and $E_3$ as defined in the above link. 

### Previously coded:

The following formulae were coded and integrated previously.

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

$$T' := \frac{x}{2} + \frac{\pi t}{8}$$

$$N := \lfloor \sqrt{ \frac{T'}{2\pi}} \rfloor$$

## New code

Expressions below are presented here in an order convenient for coding.

By convention, $z=x+iy$ and $s = \frac{1+iz}{2}$. Hence in the expressions below, $\frac{1-y+ix}{2}=s$ and $\frac{1+y+ix}{2} = \overline{1-s}$

The derivation [Effective bounds on H t - second approach](http://michaelnielsen.org/polymath1/index.php?title=Effective_bounds_on_H_t_-_second_approach) also establishes the following conventions, $s = \sigma+iT$, $T \geq T_0 \geq 10$, and $t \leq 1/2$. Thus, $T$ and $T_0$, like $t$ and $s$, are essentially parameters. Two further definitions, $T_0'$ and $\alpha_0$ are simple enough not to require coding as functions.

$$T'_0 := T_0 + \frac{\pi t}{8}$$

$$a_0 := \sqrt{\frac{T'_0}{2\pi}}$$

Since $E_1$ and $E_2$ depend on $\epsilon'(s)$, it's convenient to implement $\epsilon'(s)$ first. Its definition, below, contains terms $Re(s)$ and $Re\alpha_1(s)$ which I take to be the real part of their arguments. That would be consistent with, e.g., the corresponding Python code (function `eps_err`, lines 302-303 of `mputility.py`)

```
    alph = alpha1(s)
    term1 = sigma + (t/2.0) * (alph.real) - (t/4.0) * mp.log(N)
```

$$\epsilon'(s) := \frac{1}{2} \sum_{n=1}^N \frac{1}{n^{\mathrm{Re}(s) + \frac{t \mathrm{Re} \alpha_1(s)}{2} - \frac{t}{4} \log n}}
\exp(\frac{1}{2(T-3.33)} (\frac{t^2}{4} |\alpha_1(s) - \log n|^2 + \frac{1}{3} + t))
(\frac{t^2}{4} |\alpha_1(s) - \log n|^2 + \frac{1}{3} + t )$$

In [4]:
using DBNUpperBound
using DBNUpperBound.Asymptotics

function eps_prime(t::Ty1, T::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    ans = zero(Ty2)
    for n in 1:N
        α1 = αeff(s)
        α2 = (t^2/4)*abs(α1-log(n))^2 + 1/3 + t
        ans += 1/(real(s) + t*real(α1)/2 - t/4*log(n)) * bigexp(α2/(2*(T-3.33))) * α2
    end
    return ans/2
end

eps_prime (generic function with 1 method)

$E_1$ can now be implemented.
$$E_1 := \frac{1}{8 (T - 3.33)} \exp( \frac{t}{4} \mathrm{Re} \alpha_1(\frac{1-y+ix}{2})^2 ) |H_{0,1}(\frac{1-y+ix}{2})| \epsilon'(\frac{1-y+ix}{2})$$

In [5]:
function E1(t::Ty1, T::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    return 1/(8*(T-3.33))*bigexp(t/4*real(α(s))^2 * abs(H01(s))*eps_prime(s))
end

E1 (generic function with 1 method)

Note that the expression for $E_2$ is the same as that for $E_1$ except that $\frac{1+y+ix}{2} = \overline{1-s}$ replaces $\frac{1-y+ix}{2}=s$.

$$E_2 := \frac{1}{8 (T - 3.33)} \exp( \frac{t}{4} \mathrm{Re} \alpha_1(\frac{1+y+ix}{2})^2 ) |H_{0,1}(\frac{1+y+ix}{2})| \epsilon'(\frac{1+y+ix}{2})$$

Thus

In [6]:
function E2(t::Ty1, T::Ty1, s::Ty2, N::Int) where {Ty1<:Real, Ty2<:Number}
    return E1(t,T,(1-s)',N)
end

E2 (generic function with 1 method)

$$E_3 := \frac{1}{8} \sqrt{\pi} \exp( -\frac{t \pi^2}{64} ) (T')^{3/2} e^{-\pi T/4}  \int_{-\infty}^\infty v(\sigma) w(\sigma) f(\sigma)\ d\sigma$$

$$f(\sigma) := \frac{1}{2\sqrt{\pi t}} (e^{-(\sigma-(1-y)/2)^2/t} + e^{-(\sigma-(1+y)/2)^2/t}) \quad (4.1)$$

$$w(\sigma) := (1 + \frac{\sigma^2}{(T'_0)^2})^{1/2} (1 + \frac{(1-\sigma)^2}{(T'_0)^2})^{1/2} 
\exp( \frac{(\sigma-1)_+}{4} \log (1 + \frac{\sigma^2}{(T'_0)^2}) + (\frac{T'_0}{2} \arctan \frac{\sigma}{T'_0} - \frac{\sigma}{2}) 1_{\sigma \lt 0} + \frac{1}{12(T'_0 - 0.33)})$$

$$v(\sigma) := 1 + (0.400 \frac{9^\sigma}{a_0} + 0.346 \frac{2^{3\sigma/2}}{a_0^2}) 1_{\sigma \geq 0} + (9/10)^{\lceil -\sigma \rceil} \sum_{1 \leq k \leq 4-\sigma} (1.1)^k \frac{\Gamma(k/2)}{a_0^k} 1_{\sigma \lt 0}$$